## Code Equivalence Notebook

In [ ]:
import ast
import astpretty
import showast
import sys
import re

sys.path.insert(0, '../preprocess/')
sys.path.insert(0, '../../coarse2fine.git/src')

from sketch_generation import Sketch
from tree import SketchRepresentation
import table

In [ ]:
SKP_WORD = '<sk>'
RIG_WORD = '<]>'
LFT_WORD = '<[>'

def is_code_eq(tokens1, tokens2, not_layout=False):

    if isinstance(tokens1, SketchRepresentation):
        tokens1 = str(tokens1)
    else:
        tokens1 = ' '.join(tokens1)

    if isinstance(tokens2, SketchRepresentation):
        tokens2 = str(tokens2)
    else:
        tokens2 = ' '.join(tokens2)

    tokens1 = ['\"' if it in (RIG_WORD, LFT_WORD) else it for it in tokens1.split(' ')]
    tokens2 = ['\"' if it in (RIG_WORD, LFT_WORD) else it for it in tokens2.split(' ')]

    if len(tokens1) != len(tokens2):
        return False

    return all(map(lambda tk1, tk2: tk1 == tk2, tokens1, tokens2))

In [ ]:
# AST => Node Type [AST]
class Node:
    def __init__(self, val, *kids):
        self.val = val
        self.kids = kids
    
    def __str__(self):
        return Node.to_string(self, indent=2, c=' ')

    def __repr__(self):
        return str(self)
    
    @staticmethod
    def val_to_string(val):
        if len(val) == 1:
            n, f = val[0]
            if n == 'body':
                f = f[0]
            s = "%s: %s\n" % (n, f.__class__.__name__)
        else:
            s = ', '.join(['%s: %s' % (n, f.__class__.__name__) for n, f in val]) + "\n"
            
        return s
    
    @staticmethod
    def to_string(node, indent=2, c=' '):        
        if node.val == []:
            return ''        

        s = Node.val_to_string(node.val)

        for k in node.kids:
            _s = Node.to_string(k, indent*2)
            if _s != '':
                s += (c * indent) + _s

        return s
    
class Nil(Node):
    def __init__(self):
        self.val = None
        self.kids = []
    
    def __str__(self):
        return "x"
    
    def __repr__(self):
        return str(self)
    
class Leaf(Node):
    def __init__(self, val):
        self.val = val
        self.kids = []
    
    def __str__(self):
        return '%d' % self.val
    
    def __repr__(self):
        return str(self)
    
# TODO
    
def cons_tree(t):
    val = list(ast.iter_fields(t))
    kids = list(ast.iter_child_nodes(t))
    
    return Node(val, *[cons_tree(k) for k in kids])

def zip_tree_pred(pred, t1, t2):    
    zs = [pred(t1.val, t2.val)]
    
    for k1, k2 in zip(t1.kids, t2.kids):
        zs.append(zip_tree_pred(pred, k1, k2))
    
    return all(zs)

In [ ]:
code1 = '[x for x in range(10)]'
code2 = '[i for i in [1,2,3]]'

tree1 = ast.parse(code1)
tree2 = ast.parse(code2)

t1 = cons_tree(tree1)
t2 = cons_tree(tree2)

def cmp_func(x, y):
    s1 = Node.val_to_string(x)
    s2 = Node.val_to_string(y)
    return s1 == s2
        
zip_tree_pred(cmp_func, t1, t2)

In [ ]:
%%showast
x = self.func(1, 'test', var)

In [ ]:
%%showast
raise RuntimeError('[%s]' % self.get_err_msg(timestamp[:2]))

# tree = Node(Assign, [Attribute(Name(self), var), Name(x)])

In [ ]:
# astpretty.pprint(tree1.body[0], indent=' ' * 4)
astpretty.pprint(tree2.body[0], indent=' ' * 4)

## Eval framework

In [ ]:
## TODO